## Testing Model

In [6]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip = True)

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
x_train= train_datagen.flow_from_directory('dataset/training_set', target_size=(64,64), batch_size=300,class_mode='categorical', color_mode = "grayscale")

Found 15750 images belonging to 9 classes.


In [10]:
x_test= test_datagen.flow_from_directory('dataset/test_set', target_size=(64,64), batch_size=300,class_mode='categorical', color_mode = "grayscale")

Found 2250 images belonging to 9 classes.


In [11]:
from keras.models import Sequential

In [12]:
from keras.layers import Dense

In [13]:
from keras.layers import Convolution2D

In [14]:
from keras.layers import MaxPooling2D

In [15]:
from keras.layers import Dropout

In [16]:
from keras.layers import Flatten

In [17]:
model=Sequential()

In [18]:
model.add(Convolution2D(32, (3,3), input_shape=(64,64,1), activation = 'relu'))
#no. of feature detectors, size of featuredetector, image size, activation function

In [19]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [20]:
model.add(Flatten())

In [21]:
model.add(Dense(units=512, activation='relu'))

In [22]:
model.add(Dense(units=9, activation='softmax'))

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit_generator(x_train, steps_per_epoch=24, epochs=10, validation_data=x_test, validation_steps=40)
#steps_per_epoch = no. of train images//batch size

C:\Users\Renu Alwar\AppData\Local\Temp\ipykernel_1552\1124240008.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train, steps_per_epoch=24, epochs=10, validation_data=x_test, validation_steps=40)


Epoch 1/10
24/24 [==============================] - 27s 1s/step - loss: 0.0121 - accuracy: 0.9976 - val_loss: 0.1947 - val_accuracy: 0.9756
Epoch 2/10
24/24 [==============================] - 19s 784ms/step - loss: 0.0093 - accuracy: 0.9983
Epoch 3/10
24/24 [==============================] - 17s 698ms/step - loss: 0.0094 - accuracy: 0.9980
Epoch 4/10
24/24 [==============================] - 14s 561ms/step - loss: 0.0049 - accuracy: 0.9993
Epoch 5/10
24/24 [==============================] - 12s 476ms/step - loss: 0.0083 - accuracy: 0.9982
Epoch 6/10
24/24 [==============================] - 11s 466ms/step - loss: 0.0052 - accuracy: 0.9987
Epoch 7/10
24/24 [==============================] - 11s 452ms/step - loss: 0.0035 - accuracy: 0.9994
Epoch 8/10
24/24 [==============================] - 10s 423ms/step - loss: 0.0041 - accuracy: 0.9993
Epoch 9/10
24/24 [==============================] - 10s 430ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 10/10
24/24 [==============================] -

In [64]:
from skimage.transform import resize
from numpy import *
from keras.models import load_model
import numpy as np
import cv2

def detect(frame):
        
        model=load_model('aslpng1.h5')
        model.save('aslpng1.h5')
        img = resize(frame,(64,64,1))
        img = np.expand_dims(img,axis=0) 
        if(np.max(img)>1):
            img = img/255.0
        prediction = model.predict(img)
        print(prediction)
#         prediction = (model.predict_classes(img))
        predict_x=model.predict(img) 
        classes_x=np.argmax(predict_x,axis=1)
        print(classes_x)
frame = cv2.imread(r"C:\Users\Renu Alwar\Project\dataset\test_set\I\1.png")
data = detect(frame)

[[0.10409457 0.11796871 0.09888812 0.10719631 0.13776585 0.10792505
  0.11153119 0.11200681 0.10262337]]
[4]


C:\anaconda\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
import numpy as np
import cv2
import os
from keras.models import load_model
from flask import Flask, render_template, Response
from tensorflow.keras.models import Sequential
import tensorflow as tf
from gtts import gTTS
global graph
global writer
from skimage.transform import resize

# graph=tf.get_default_graph()
graph= tf.compat.v1.get_default_graph()
writer = None
model = load_model('aslpng1.h5')
vals = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
app = Flask(__name__)

# tf.compat.v1.disable_eager_execution()
# print(tf.compat.v1.get_default_graph())

print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(0)
pred = ""
@app.route('/')
def index():
    return render_template('index.html')

def detect(frame):
    img = resize(frame,(64,64,1))
    img = np.expand_dims(img,axis=0)
    if(np.max(img)>1):
        img = img/255.0
    with graph.as_default():
            prediction = model.predict_classes(img)
    print(prediction)
    pred=vals[prediction[0]]
    print(pred)
    return pred

@app.route('/video feed')
def video_feed():
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True,use_reloader=False)
    


C:\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[INFO] accessing video stream...
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://192.168.29.183:5000/ (Press CTRL+C to quit)
192.168.29.183 - - [17/Nov/2022 21:48:43] "GET / HTTP/1.1" 200 -
192.168.29.183 - - [17/Nov/2022 21:48:43] "GET /favicon.ico HTTP/1.1" 404 -
